In [20]:
# 한국인 강의음성 전처리 및 csv 파일 생성
# https://aihub.or.kr/aidata/30708/download

# 압축을 풀고 "한국어 강의 음성" 폴더의 이름을 KlecSpeech로 변경한다.
# 추가 히스토리
# 2022-06-15 array 추가
# 2022-06-17 정규식 교체 

In [21]:
import re

def preprocessing(text):
    # 괄호(문자)/(발음) 일 때, 발음만 살리는 코드
    """
    괄호 및 괄호 안에 있는 문장을 모두 벗겨내기
    \([^\)]*\)\/|\(|\)

    조건 1: (문장)/ 을 삭제한다. 
    \([^\)]*\)    (사과)/(배) (1000원)/(천원)
    \/            뒤에 /를 추가하면 뒤에 /가 없는 (배) 와 (천원)은 조건 탈락

    |             조건 1과 조건 2를 OR로 검출

    조건 2 :  ( 과 )를 검출
    \(|\)  
    """  
    re_text = re.sub(r'[+?!]', '', text)
    re_text = re.sub(r'o/', '', re_text)
    re_text = re.sub(r"u/","<unk>",  re_text)
    re_text = re.sub(r"b/","<b>",  re_text)
    re_text = re.sub(r"n/","<n>",  re_text)
    re_text = re.sub(r"l/","<|>",  re_text)
    re_text = re.sub(r'\([^\)]*\)\/|\(|\)', '', re_text)
    re_text = re.sub(r'[/]', '', re_text)

    return re_text

test_text = "la/ l/ 와/ 와// b/ 안녕하세요. n/ na/ l/ (사과)/(배)는 (1000원)/(천원) 입니다?!+ u/ o/"
test_ret = preprocessing(test_text)
print(test_ret)

la <|> 와 와 <b> 안녕하세요. <n> na <|> 배는 천원 입니다 <unk> 


## 오디오 데이터 추가

In [23]:
import json
import os
import glob
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import librosa

In [24]:
text_dict = {'filename':[],'text':[],'array':[]}

path='./dataset/KlecSpeech/Validation/'

# path = Path('./한국어 강의 음성/Validation/')

# wav 파일 기준으로 text 파일이름이 동일, 확장자는 wav, txt로 각각 다름
files = [f for f in glob.glob(path + "**/*.wav", recursive=True)]

# 테스트 코드
# files = files[0:3]
# print(files)
import time

start = time.time()
print("This time is being calculated")

for i in tqdm(range(len(files))):
    txt_file = files[i].replace(".wav", ".txt")
    files[i] = files[i].replace("\\", "/")

#     print(txt_file)
    if not os.path.exists(txt_file):
        print("The file does not exist", txt_file)
    
    with open(txt_file ,'r',encoding='utf8') as f:
        text = f.read()
        
        # 텍스트 전처리
        text = preprocessing(text)

        # 항목들 값을 저장 \
        text_dict['filename'].append(files[i]) # filename
        text_dict['text'].append(text)
        
        # 각 wav파일을 16000으로 나눠서 저장
        array,_ = librosa.load(files[i],sr=16000)
        text_dict['array'].append(list(array))

    if i%500==499:
        n=i//500
        df = pd.DataFrame(text_dict)
        df.to_csv(f'klecspeech_ko_500_{n:03d}.csv',index=False,sep='\t')
        print(f'{n:03d} complete')

        # text_dict을 초기화
        text_dict = {'filename':[],'text':[],'array':[]}
        
# df = pd.DataFrame(text_dict)
# df.head()

print(time.time() - start)

This time is being calculated


  2%|█▏                                                                          | 507/32971 [00:35<8:56:32,  1.01it/s]

000 complete


  3%|██▎                                                                        | 1007/32971 [01:13<8:56:59,  1.01s/it]

001 complete


...

063 complete


 99%|██████████████████████████████████████████████████████████████████████████▉ | 32517/32971 [36:46<02:26,  3.10it/s]

064 complete


100%|████████████████████████████████████████████████████████████████████████████| 32971/32971 [36:53<00:00, 14.90it/s]

2213.407388448715


In [ ]:
csv_df = pd.read_csv('klecspeech_ko_000.csv',sep='\t')
csv_df.head()

In [ ]:
for i in range(100):
    print(text_dict['src'][i], text_dict['text'][i])

In [ ]:
df.to_csv('order_speech_ko.csv')

In [ ]:
df['text'].unique()
# df.read_csv('order_speech_ko.csv')
# result = [i for i in my_list if i]

In [ ]:
df.info()

In [ ]:
df.describe()

## 텍스트 누락 확인

In [ ]:
df['text'].min()

In [ ]:
df['text_len'] = df['text'].map(len)

df[['text_len']].idxmin()

In [ ]:
df[['text_len']].idxmin()

In [ ]:
df['text'][22410]

In [ ]:
print(df['src'][22410])

In [ ]:
# 찾아보면 #NAME이라고 되어 있음.